In [1]:
import importlib
import sys
sys.path.append('../src')

import qa
importlib.reload(qa)
import qa

import plot
from plot import plot_from_file

import pickle
import numpy as np
import pandas as pd

import re
import random
import ast

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_rel
from scipy.stats import pearsonr

In [9]:
with open(f'../data/MMLU-Pro/sample_results/grp_count.pkl', 'rb') as f:
    temp = pickle.load(f)

In [6]:
benchmarks = ["MMLU-Pro", "GPQA-Diamond", "ARC", "OpinionQA", "GlobalOpinionQA", "SIQA"]
factual = ["MMLU-Pro", "GPQA-Diamond", "ARC"]
opinion = ["OpinionQA", "GlobalOpinionQA", "SIQA"]

for benchmark in benchmarks:

    with open(f'../data/{benchmark}/sample_results/org.pkl', 'rb') as f:
        res_org = pickle.load(f)

    with open(f'../data/{benchmark}/sample_results/grp_count.pkl', 'rb') as f:
        previous_results = pickle.load(f)

    input_feat_list = []
    for disagree_size in range(1, 11): # row
        input_row = []
        for agree_size in range(1, 11): # column

            if agree_size <= 5 and disagree_size <= 5:
                continue

            
            eval_feat = {
                'type': 'grp_count',
                'agree_size': agree_size,
                'disagree_size': disagree_size,
                'disagree_type': '2nd',
                'q_type': 'factual' if benchmark in factual else 'opinion',
                'order': 'random'
            }
            input_row.append(eval_feat)
        input_feat_list.append(input_row)

    results, accuracy = qa.qa_eval_matrix(res_org, input_feat_list)

    results = previous_results + results

    with open(f'../data/{benchmark}/sample_results/grp_count.pkl', 'wb') as f:
        pickle.dump(results, f)


Processing grp_count samples...


Processing:   0%|          | 1/15000 [00:00<3:20:35,  1.25it/s]


KeyboardInterrupt: 